In [62]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Access the OpenAI key
openai_key = os.getenv("OPENAI_API_KEY")
activeloop_key = os.getenv("ACTIVELOOP_TOKEN")


In [63]:
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.deeplake import DeepLake
from langchain.document_loaders import PyPDFLoader
import random
from langchain.memory import ConversationBufferWindowMemory
from langchain.embeddings import OpenAIEmbeddings
from tqdm import tqdm
from typing import List

from langchain.chains.openai_functions import (
    create_structured_output_chain,
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, SystemMessage
from pydantic import BaseModel, Field
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

In [64]:
openai_embeddings = OpenAIEmbeddings()

In [65]:
# dataset_path = "hub://101010/text_embedding"
# # Create a DeepLake instance and add the documents
# db = DeepLake.from_documents(documents, dataset_path=dataset_path, embedding=OpenAIEmbeddings())

db = DeepLake(
    dataset_path=f"hub://siddartha10/manufacturing_CSI_2048",  # org_id stands for your username or organization from activeloop
    embedding=openai_embeddings,
    runtime={"tensor_db": True},
    token=activeloop_key,
    overwrite=True, # user overwrite flag if you want to overwrite the full dataset
    read_only=False,
)

Your Deep Lake dataset has been successfully created!


In [66]:
# Replace 'file_paths' with the paths to your local PDF files
file_paths = [
    "manual\'s.pdf",
    "Operation and Maintenance Manual.pdf",
    "Operations Manual.pdf"
]
# Initialize an empty list to store all pages
pages = []

# Loop through each manual and load its pages
for manual_path in file_paths:
    loader = PyPDFLoader(manual_path)
    pages1 = loader.load_and_split()
    pages.extend(pages1)

In [68]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_size = 1048
chunk_overlap = 200
docs_new = []

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,chunk_overlap=chunk_overlap
)

for doc in pages:
    if len(doc.page_content) < chunk_size:
        docs_new.append(doc)
    else:
        docs = text_splitter.create_documents([doc.page_content])
        docs_new.extend(docs)

In [69]:
len(docs_new)

387

In [70]:
docs = db.add_documents(docs_new)

Creating 387 embeddings in 1 batches of size 387:: 100%|██████████| 1/1 [00:09<00:00,  9.54s/it]

Dataset(path='hub://siddartha10/manufacturing_CSI_2048', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype       shape      dtype  compression
  -------    -------     -------    -------  ------- 
   text       text      (387, 1)      str     None   
 metadata     json      (387, 1)      str     None   
 embedding  embedding  (387, 1536)  float32   None   
    id        text      (387, 1)      str     None   


In [71]:
# fetch dataset docs and ids if they exist (optional you can also ingest)
docs = db.vectorstore.dataset.text.data(fetch_chunks=True, aslist=True)["value"]
ids = db.vectorstore.dataset.id.data(fetch_chunks=True, aslist=True)["value"]

In [72]:
len(ids)

387

In [73]:
# If we pass in a model explicitly, we need to make sure it supports the OpenAI function-calling API.
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


class Questions(BaseModel):
    """Identifying information about the manufacturing system."""

    question: str = Field(..., description="Questions about manufacturing system")


prompt_msgs = [
    SystemMessage(
        content="You are a world class expert for generating questions based on provided context. \
                You make sure the question can be answered by the text."
    ),
    HumanMessagePromptTemplate.from_template(
        "Use the given text to generate a question from the following input: {input}"
    ),
    HumanMessage(content="Tips: Make sure to answer in the correct format"),
]
prompt = ChatPromptTemplate(messages=prompt_msgs)
chain = create_structured_output_chain(Questions, llm, prompt, verbose=True)

text = """
UWM CSI Vial Filling Connected Smart Manufacturing (CSM) System The UWM CSI Vial Filling CSM  System is an intelligent manufacturing system using the latest Industry 4.0 connected advanced manufacturing equipment and techniques to produce  vials filled with varying product using a variety of filling methods and capturing process data that can be used for data analysis and system optimization.  The  Vial Filling CSM System is a platform that university faculty and students will use for both education and research to  further the advancement of a connected enterprise.   The CAM components are  integrate d seamlessly within a fully  integrated architecture and connected  enterprise using cutting -edge smart -data devices at all layers.     The process overview detailed in this section describes in general how the Vial Filling Connected Smart Manufacturing system functions as a complete system processing components.  Operational steps and slight variances in the process may differ from what is described here depending on the configuration parameters or  using the stations  in a dry cycle mode .  The Vial Filling CSM utilizes localized system configuration setting within the machine as well as process data requirements and parameters to determine how  to process the product and which stations and inspections are required to complete the production job.  When running under MES Production Center control, the process requirements and parameters are received  from the MES system as to where  and how to process and inspect the vials produced by the system .    """
questions = chain.run(input=text)
print(questions)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a world class expert for generating questions based on provided context.                 You make sure the question can be answered by the text.
Human: Use the given text to generate a question from the following input: 
UWM CSI Vial Filling Connected Smart Manufacturing (CSM) System The UWM CSI Vial Filling CSM  System is an intelligent manufacturing system using the latest Industry 4.0 connected advanced manufacturing equipment and techniques to produce  vials filled with varying product using a variety of filling methods and capturing process data that can be used for data analysis and system optimization.  The  Vial Filling CSM System is a platform that university faculty and students will use for both education and research to  further the advancement of a connected enterprise.   The CAM components are  integrate d seamlessly within a fully  integrated architecture and connected  enterprise using cutting -

In [74]:
def generate_queries(docs: List[str], ids: List[str], n: int=100 ):
    questions = []
    relevances = []
    pbar = tqdm(total=n)
    while len(questions) < n:
        # 1. randomly draw a piece of text and relevance id
        r = random.randint(0, len(docs) - 1)
        text, label = docs[r], ids[r]

        # 2. generate queries and assign and relevance id
        generated_qs = [chain.run(input=text).question]
        questions.extend(generated_qs)
        relevances.extend([[(label, 1)] for _ in generated_qs])
        pbar.update(len(generated_qs))
        if len(questions) % 10 == 0:
            print(f"q: {len(questions)}")
    return questions[:n], relevances[:n]


chain = create_structured_output_chain(Questions, llm, prompt, verbose=False)
questions, relevances = generate_queries(docs, ids, n=1000)

  1%|          | 10/1000 [00:12<21:25,  1.30s/it]

q: 10


  2%|▏         | 20/1000 [00:27<24:56,  1.53s/it]

q: 20


  3%|▎         | 30/1000 [00:40<19:01,  1.18s/it]

...

q: 980


 99%|█████████▉| 990/1000 [21:38<00:11,  1.19s/it]

q: 990


100%|██████████| 1000/1000 [21:50<00:00,  1.31s/it]

q: 1000


In [75]:
train_questions, train_relevances = questions[:800], relevances[:800]
test_questions, test_relevances = questions[800:], relevances[800:]

In [76]:
job_id = db.vectorstore.deep_memory.train(
    queries=train_questions,
    relevance=train_relevances,
)

Starting DeepMemory training job
Your Deep Lake dataset has been successfully created!


Preparing training data for deepmemory:


Creating 800 embeddings in 2 batches of size 500:: 100%|██████████| 2/2 [00:17<00:00,  8.69s/it]


DeepMemory training job started. Job ID: 65724b714730ecbeae7c9c90


In [83]:
db.vectorstore.deep_memory.status(job_id)

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/siddartha10/manufacturing_CSI_2048
--------------------------------------------------------------
|                  65724b714730ecbeae7c9c90                  |
--------------------------------------------------------------
| status                     | completed                     |
--------------------------------------------------------------
| progress                   | eta: 2.0 seconds              |
|                            | recall@10: 98.12% (+9.38%)    |
--------------------------------------------------------------
| results                    | recall@10: 98.12% (+9.38%)    |
--------------------------------------------------------------




In [84]:
recall = db.vectorstore.deep_memory.evaluate(
    queries=test_questions,
    relevance=test_relevances,
)

Embedding queries took 1.96 seconds
---- Evaluating without Deep Memory ---- 
Recall@1:	  63.0%
Recall@3:	  82.0%
Recall@5:	  86.0%
Recall@10:	  90.0%
Recall@50:	  98.0%
Recall@100:	  99.0%
---- Evaluating with Deep Memory ---- 
Recall@1:	  74.0%
Recall@3:	  91.5%
Recall@5:	  95.0%
Recall@10:	  99.5%
Recall@50:	  100.0%
Recall@100:	  100.0%


In [85]:
retriver = db.as_retriever()
retriver.search_kwargs["deep_memory"] = True
retriver.search_kwargs["k"] = 10

In [98]:
memory = ConversationBufferWindowMemory(
        memory_key='chat_history', return_messages=True, window_size=10)

In [108]:
conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=retriver,
        memory=memory,
    )

ValidationError: 1 validation error for ConversationalRetrievalChain
memory_key
  extra fields not permitted (type=value_error.extra)

In [100]:
conversation_chain.run("hey")

'Hello! How can I assist you today?'

In [101]:
conversation_chain.run("can you include all of them?")

"I'm sorry, but I don't have access to the previous conversation. Could you please provide me with the specific question you would like me to answer?"

In [102]:
conversation_chain.run("there are only 6 stations but there should be 9?")

'There should be 9 stations in total.'

In [104]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='hey'),
  AIMessage(content='Hello! How can I assist you today?'),
  HumanMessage(content='can you include all of them?'),
  AIMessage(content="I'm sorry, but I don't have access to the previous conversation. Could you please provide me with the specific question you would like me to answer?"),
  HumanMessage(content='there are only 6 stations but there should be 9?'),
  AIMessage(content='There should be 9 stations in total.')]}

In [151]:

from langchain.memory import ConversationSummaryMemory
memory = ConversationSummaryMemory(
    llm=llm, memory_key="chat_history", return_messages=True
)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriver, memory=memory)

In [152]:
print(qa.run("my name is siddu"))

I am an AI language model and do not have a personal name. You can refer to me as OpenAI Assistant. How can I assist you today?


In [153]:
print(qa.run("what is my name?"))

The given context does not provide any information about the name of a specific human.


In [115]:
print(qa.run("there are 9 stations"))

There are 9 stations.


In [116]:
print(qa.run("in the above how many stations are there?"))

There are 9 stations mentioned in the above conversation.


In [117]:
memory.load_memory_variables({})

{'chat_history': [SystemMessage(content='The human introduces themselves as Siddu and the AI responds as OpenAI Assistant. The AI asks how it can assist the human. The human mentions that there are 9 stations and the AI simply repeats the statement. When asked about the number of stations mentioned, the AI confirms that there are 9 stations mentioned in the above conversation.')]}

In [118]:
qa.run("what is my name?")

'The provided context does not mention the name of any specific human.'

In [119]:
memory.load_memory_variables({})

{'chat_history': [SystemMessage(content='The human introduces themselves as Siddu and the AI responds as OpenAI Assistant. The AI asks how it can assist the human. The human mentions that there are 9 stations and the AI simply repeats the statement. When asked about the number of stations mentioned, the AI confirms that there are 9 stations mentioned in the above conversation. When the human asks about their name, the AI responds that the provided context does not mention the name of any specific human.')]}

In [120]:
qa.run("what is the human name?")

'The given context does not provide any information about the name of a specific human.'

In [129]:
def get_chat_history(inputs) -> str:
    res = []
    for human, ai in inputs:
        res.append(f"Human:{human}\nAI:{ai}")
    return "\n".join(res)


qa = ConversationalRetrievalChain.from_llm(
    llm, retriver, get_chat_history=get_chat_history
)

In [135]:
print(result["answer"])

That's nice to know, Siddu! How can I assist you today?


In [144]:
def get_chat_history(inputs) -> str:
    print(inputs)
    res = []
    for human, ai in inputs:
        res.append(f"Human:{human}\nAI:{ai}")
    return "\n".join(res)
qa = ConversationalRetrievalChain.from_llm(
    llm, retriver, get_chat_history=get_chat_history
)
chat_history = []
result = qa({"question": "my name is siddu", "chat_history": chat_history})
chat_history.append((result["question"], result["answer"]))

[]


In [140]:
print(result["answer"])

That's nice to know, Siddu! How can I assist you today?


In [149]:
result = qa({"question": "what was the human name", "chat_history": chat_history})
chat_history.append((result["question"], result["answer"]))

[('my name is siddu', "That's nice to know, Siddu! How can I assist you today?"), ('what was my name', "I'm sorry, but I don't have access to that information."), ('what was the user name', "The provided context does not mention the user's name.")]


In [150]:
print(result["answer"])

The provided context does not mention the name of any specific human.


In [143]:
print(chat_history)

[('my name is siddu', "That's nice to know, Siddu! How can I assist you today?"), ('what was my name', "I'm sorry, but I don't have access to that information.")]


In [180]:
from langchain.memory import ConversationBufferMemory
memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history", return_messages=True,max_tokens=100)

In [181]:
qa = ConversationalRetrievalChain.from_llm(llm, retriver, memory=memory)

In [182]:
query = "what are the different stations"
result = qa({"question": query})

In [183]:
print(result["answer"])

The different stations mentioned in the provided context are:

1) Station 1 - Unload
2) Station 2 - Dry Fill
3) Station 3 - Wet Fill (Single Nozzle Sequential)
4) Station 4 - Wet Fill (Single Nozzle Mixed)
5) Station 5 - Wet Fill (Individual)
6) Station 6 - Capping
7) Station 7 - Vision Inspection
8) Station 8 - Weigh Check
9) Station 9 - Inspect/Repair


In [184]:
query = "Explain"
result = qa({"question": query})

In [185]:
print(result["answer"])

Yes, I can explain the different stations mentioned in the context.

1. Station 1 - Load/Unload: This station is responsible for loading and unloading vials onto the MagneMotion transport system. It includes steps such as inspecting vial position, barcode reading, and transferring vial data.

2. Station 2 - Dry Fill: This station involves the use of a robot to perform dry filling operations. It includes steps such as sending process data to the robot and triggering the robot to run.

3. Station 3 - Wet Fill (Single Nozzle Sequential): This station is used for wet filling operations. It consists of a single nozzle that dispenses specified quantities of fluid into the vial in a sequential manner.

4. Station 4 - Wet Fill (Single Nozzle Mixed): Similar to Station 3, this station also performs wet filling operations. However, it uses a single nozzle to dispense a mixture of fluids into the vial.

5. Station 5 - Wet Fill (Individual): This station has four separate nozzles that are used to 

In [164]:
query = "i am siddu working at csi station 1 and my firsnd eswar working at station 2 and ethical hacker rishi is wokring at home"
result = qa({"question": query})

In [165]:
print(result["answer"])

Based on the provided context, there is no information about who is working at home or their names.


In [166]:
query = "who is working at home and what is his name"
result = qa({"question": query})

In [167]:
print(result["answer"])

Based on the provided context, there is no information about who is working at home or their names.


In [193]:
prompt = memory.load_memory_variables({})['chat_history'][0].content

In [196]:
len(prompt)

506

In [194]:
query = "what is the previous question about"
result = qa({"question": prompt+query})

In [195]:
print(result["answer"])

The previous question is regarding the description and setup of various stations and sequences in the CSM Operations Manual.


In [199]:
prompt = memory.load_memory_variables({})['chat_history'][0].content
query = "my name is siddu"
result = qa({"question": prompt+query})

In [200]:
print(result["answer"])

The user is inquiring about a specific aspect or detail of the LDS system. However, the specific question or topic is not mentioned in the given context.
